# 03. RAGの概念デモ (Retrieval-Augmented Generation)

このノートブックでは、LLMのハルシネーション（もっともらしい嘘）を抑制し、事実に基づいた回答を生成させるための強力な技術、**RAG (Retrieval-Augmented Generation)** の基本的な概念を学びます。

RAGは、以下の3ステップで構成されます。

1.  **検索 (Retrieve)**: ユーザーの質問に関連する情報を、信頼できる知識源（ドキュメントなど）から探し出す。
2.  **拡張 (Augment)**: 探し出した情報をプロンプトに埋め込み、元の質問と一緒にLLMに渡す。
3.  **生成 (Generate)**: LLMは、与えられた情報（コンテキスト）を**最優先の参考資料として**、質問に対する回答を生成する。

これにより、LLMが元々持っていなかった知識についても、正確な回答ができるようになります。

## 事前準備

`00_setup_common.ipynb` を実行して、ライブラリのインストールと設定が完了していることを確認してください。

In [ ]:
# 編集禁止セル
# セットアップの確認と共通モジュールのインポート
import os
import sys
import torch
from google.colab import drive

if not os.path.isdir('/content/drive'): drive.mount('/content/drive')
repo_path = '/content/llm_lab'
if not os.path.exists(repo_path):
    !git clone https://github.com/akio-kobayashi/llm_lab.git
os.chdir(repo_path)

# 必要なライブラリのインストール
!pip install -q -U transformers==4.41.2 accelerate==0.30.1 bitsandbytes==0.43.1 sentence-transformers==2.7.0 faiss-gpu==1.7.2 peft==0.10.0 trl==0.8.6 datasets==2.19.0 gradio==4.31.5
if 'src' not in sys.path: sys.path.append(os.path.abspath('src'))

try:
    from src.common import load_llm, generate_text
    print('共通モジュールのインポートが完了しました。')
except ImportError:
    print('共通モジュールのインポートに失敗しました。')

## モデルのロード

演習で共通して使用するLLMをロードします。

In [ ]:
# 編集禁止セル
model, tokenizer = None, None
try:
    model, tokenizer = load_llm(use_4bit=True)
except Exception as e:
    print(f'モデルのロード中にエラーが発生しました: {e}')

## RAGなし vs RAGあり の比較

`data/docs/anime_docs_sample.jsonl` に含まれる架空のアニメ『星屑のメモリー』に関する質問をしてみます。
この情報はLLMの学習データには含まれていないため、RAGなしでは答えることができません。

### ケース1: RAGなし (LLM単体)

まずは、これまで通りLLMに直接質問してみます。高い確率でハルシネーションが発生します。

In [ ]:
# --- ここを編集 --- #
question = "アニメ『星屑のメモリー』の主人公の名前と、彼が乗る機動兵器の名前を教えてください。"
# --- 編集ここまで --- #

# 編集禁止セル
if model and tokenizer:
    prompt_no_rag = f"以下は、タスクを説明する指示です。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n{question}\n\n### 応答:\n"
    
    generated_text = generate_text(model, tokenizer, prompt_no_rag)
    
    print("--- 質問 ---")
    print(question)
    print("\n--- RAGなしの回答 ---")
    answer = generated_text.split("### 応答:")[-1].strip()
    print(answer)
else:
    print("モデルがロードされていません。")

### ケース2: RAGあり (手動でコンテキストを付与)

次に、RAGの「検索(Retrieve)」と「拡張(Augment)」のステップを**手動で**行ってみます。

1.  **検索**: `anime_docs_sample.jsonl` の中から、質問に関係しそうな部分を事前に探し出しておきます。（このセルでは、あらかじめ用意したテキストを `context_document` 変数に入れています）
2.  **拡張**: その情報をプロンプトに埋め込みます。

In [ ]:
# 編集禁止セル

# 1. 検索(Retrieve)ステップ：手動で関連ドキュメントを用意
context_document = """
doc_id: stardust_memory
title: 星屑のメモリー
section: 概要
text: 『星屑のメモリー』は、銀河連邦とザイオ帝国との間で繰り広げられる壮大な宇宙戦争を描いたSFアニメ作品。主人公の少年カイ・ミナトが、伝説の機動兵器「スターダスト」と出会い、運命の渦に巻き込まれていく物語。友情、裏切り、そして成長をテーマに、緻密なメカニック描写と重厚な人間ドラマが特徴。
"""

# 2. 拡張(Augment)ステップ：プロンプトにコンテキストを埋め込む
prompt_with_rag = f"""以下は、参考情報と、それに基づいた質問です。参考情報を使って、質問に正確に答えてください。

### 参考情報:
{context_document}

### 質問:
{question}

### 応答:
"""

print("--- RAGで拡張されたプロンプト ---")
print(prompt_with_rag)

この拡張されたプロンプトをLLMに渡して、回答を生成させてみましょう。
LLMは「参考情報」の内容を元に回答するため、事実に基づいた正確な答えが返ってくるはずです。

In [ ]:
# 編集禁止セル
# 3. 生成(Generate)ステップ
if model and tokenizer:
    generated_text = generate_text(model, tokenizer, prompt_with_rag)
    
    print("--- 質問 ---")
    print(question)
    print("\n--- RAGありの回答 ---")
    answer = generated_text.split("### 応答:")[-1].strip()
    print(answer)
else:
    print("モデルがロードされていません。")

## まとめ

このノートブックでは、RAGの基本的な概念を学びました。

- **RAGなし**: LLMは知らない情報について質問されると、もっともらしい嘘をつく（ハルシネーション）。
- **RAGあり**: プロンプトに外部の知識（コンテキスト）を与えることで、LLMはその情報に基づいて正確な回答を生成できる。

今回は手動で「検索」を行いましたが、実際のアプリケーションではこの部分を自動化する必要があります。
次の `04_rag_faiss_exercise.ipynb` では、`sentence-transformers` と `faiss` というライブラリを使って、質問内容に応じて動的に関連ドキュメントを検索する本格的なRAGシステムを構築します。

### メモリ解放
次のノートブックに進む前に、GPUメモリを解放します。

In [ ]:
# 編集禁止セル
import gc
if 'model' in locals() and model is not None: del model
if 'tokenizer' in locals() and tokenizer is not None: del tokenizer
model = None
tokenizer = None
gc.collect()
torch.cuda.empty_cache()
print("モデルを解放し、GPUキャッシュをクリアしました。")